In [8]:
class BlitManager:
    def __init__(self, canvas, animated_artists=()):
        """
        Parameters
        ----------
        canvas : FigureCanvasAgg
            The canvas to work with, this only works for sub-classes of the Agg
            canvas which have the `~FigureCanvasAgg.copy_from_bbox` and
            `~FigureCanvasAgg.restore_region` methods.

        animated_artists : Iterable[Artist]
            List of the artists to manage
        """
        self.canvas = canvas
        self._bg = None
        self._artists = []

        for a in animated_artists:
            self.add_artist(a)
        # grab the background on every draw
        self.cid = canvas.mpl_connect("draw_event", self.on_draw)

    def on_draw(self, event):
        """Callback to register with 'draw_event'."""
        cv = self.canvas
        if event is not None:
            if event.canvas != cv:
                raise RuntimeError
        self._bg = cv.copy_from_bbox(cv.figure.bbox)
        self._draw_animated()

    def add_artist(self, art):
        """
        Add an artist to be managed.

        Parameters
        ----------
        art : Artist

            The artist to be added.  Will be set to 'animated' (just
            to be safe).  *art* must be in the figure associated with
            the canvas this class is managing.

        """
        if art.figure != self.canvas.figure:
            raise RuntimeError
        art.set_animated(True)
        self._artists.append(art)

    def _draw_animated(self):
        """Draw all of the animated artists."""
        fig = self.canvas.figure
        for a in self._artists:
            fig.draw_artist(a)

    def update(self):
        """Update the screen with animated artists."""
        cv = self.canvas
        fig = cv.figure
        # paranoia in case we missed the draw event,
        if self._bg is None:
            self.on_draw(None)
        else:
            # restore the background
            cv.restore_region(self._bg)
            # draw all of the animated artists
            self._draw_animated()
            # update the GUI state
            cv.blit(fig.bbox)
        # let the GUI event loop process anything it has to do
        cv.flush_events()

In [23]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

blit = terminal_output = annotate_output = False

def wave(n):
    l = 10
    x, y = np.ogrid[-l:l:n * 1j,-l:l:n * 1j]
    k, omega, phi0 = 1, 1, 0

    t0 = 0
    dt = 0.1
    
    i = 0
    while True:
        i += 1
        t = t0 + i * dt
        r = np.sqrt(x * x + y * y)
        z = np.sin(k * r - omega * t + phi0)# / r

        yield i, z

def random(n):
    i = 0
    data = np.random.rand(2, n, n) < 0.5
    
    while True:
        i += 1
        yield i, data[i % 2]

def random1(n):
    i = 0
    data = 2 * np.random.randint(2, size=(n, n)) - 1
    ydata = np.random.random_sample(size=n)
    
    while True:
        i += 1
        i_, j, k = np.random.randint(n, size=3)
        data[i_, j] = - data[i_, j]
        ydata = np.random.rand()

        yield i, data, ydata


step = random1(n)
n = 100
blit = True
terminal_output = True
# annotate_output = True

# ----

i, data, ydata = next(step)

plt.close('all')
# make a new figure
fig = plt.figure()
# add a image subplot
ax = plt.subplot(211)
im = ax.imshow(data, animated=blit)
# add a line subplot
ax = plt.subplot(212)
ln, = ax.plot(ydata, animated=blit)

artists = [im, ln]
if annotate_output:
    # add a text
    text = ax.annotate(
        "0",
        (0, 1),
        xycoords="figure fraction",
        xytext=(10, -10),
        textcoords="offset points",
        ha="left",
        va="top",
        family='monospace',
        animated=blit,
    )
    artists.append(text)
if blit:
    bm = BlitManager(fig.canvas, artists)

# make sure our window is on the screen and drawn
plt.show(block=False)
plt.pause(.1)

from timeit import default_timer as timer
from IPython.display import clear_output, Markdown, HTML
freq = None
# когда окно не закрыто
while plt.fignum_exists(fig.number):
    total_start = start = timer()
#     plt.pause(.1)
    i, data, ydata = next(step)
    end = timer()
    simulation_time = end - start

    start = timer()
    # update the artists
    im.set_data(data)
    ln.set_ydata(ydata)
    if blit:
        # tell the blitting manager to do its thing
        bm.update()
    else:
        # re-drawing the figure
        fig.canvas.draw()
        # flush any pending GUI events, re-painting the screen if needed
        fig.canvas.flush_events()
        
    total_end = end = timer()
    draw_time = end - start
    total_time = total_end - total_start

#     plt.pause(.1)
    freq_ = 1 / total_time  # fps
    theta = 0.01

    if freq is None:
        freq = freq_
    else:
        freq = freq_ * theta + freq * (1 - theta)
    
    str_text = (
            f"i = {i:d}\n"
            f"blit {'on' if blit else 'off':s}\n"
#             f"freq:{fps:f} $\\frac {{1}} {{\\mathrm{{sec}}}}$\n"
            f"freq (fps):\t{freq:f} sec^-1\n"
            f"total time:\t{total_time:g} sec\n"
            f"simu time:\t{simulation_time / total_time:g}%\n"
            f"draw time:\t{draw_time / total_time:g}%"
    ).expandtabs()

    if terminal_output:
        clear_output(wait=True)
#         display(Markdown(f'```\n{str_text}\n```'))
        display(HTML(f'<code>{str_text}</code>'))
    if annotate_output:
#         text.set_text(str_text)
        text.set_text(str_text)

In [ ]:
class Plot:
    def __init__(self):
        # make a new figure
        fig = plt.figure()
        # add a image subplot
        ax = plt.subplot(211)
        im = ax.imshow(data, animated=blit)
        # add a line subplot
        ax = plt.subplot(212)
        ln, = ax.plot(ydata, animated=blit)

        artists = [im, ln]
        if annotate_output:
            # add a text
            text = ax.annotate(
                "0",
                (0, 1),
                xycoords="figure fraction",
                xytext=(10, -10),
                textcoords="offset points",
                ha="left",
                va="top",
                family='monospace',
                animated=blit,
            )
            artists.append(text)
        if blit:
            bm = BlitManager(fig.canvas, artists)
        
        # make sure our window is on the screen and drawn
        plt.show(block=False)
        plt.pause(.1)

        self._update()

In [5]:
"""Minimum code to display an image with pyqtgraph(To)"""
import sys

import numpy as np
from PyQt5 import QtWidgets
import pyqtgraph as pg

#Sample array
data = np.random.normal(size=(200, 200))
data[40:80, 40:120] += 4
data = pg.gaussianFilter(data, (15, 15))
data += np.random.normal(size=(200, 200)) * 0.1

#GUI control object
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
#Window creation
window = pg.GraphicsLayoutWidget()
#Image object creation&Set image
image = pg.ImageItem()
image.setImage(data)
#Create a box to store images&Set image object
view_box = pg.ViewBox()
view_box.addItem(image)
#Plot object creation&View created above_set box
plot = pg.PlotItem(viewBox=view_box)
#Add plot to window
window.addItem(plot)
#Window display
window.show()
#The end of the program
app.exec_()

0

In [26]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
        
    
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        n = self.n = 1000
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()

        #### Start  #####################
        self.run = True
        threading.Thread(target=self.simulation_loop).start()
#         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
        
#         self.simulation_loop()
    
    def closeEvent(self, event):
        self.run = False

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(self.tx)

    def simulation_loop(self):
        while self.run:
#             print('update')
            i, j, k = np.random.randint(self.n, size=3)
            self.data[i, j] = - self.data[i, j]
            self.ydata[k] = np.random.rand()
#             time.sleep(.001)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.9 + freq2 * 0.1
            self.tx = f'Simulation steps per second:  {self.freq:.3f}'
            print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
#             QtCore.QTimer.singleShot(1, self.simulation_loop)
            self.counter += 1


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

Simulation steps per second:  13333.876

/tmp/ipykernel_305374/1883293687.py:58: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [26]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
        
    
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        n = self.n = 1000
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()

        #### Start  #####################
        self.run = True
#         threading.Thread(target=self.simulation_loop).start()
#         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
        
        self.simulation_loop()
    
    def closeEvent(self, event):
        self.run = False

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(self.tx)

    def simulation_loop(self):
#         while self.run:
#             print('update')
            i, j, k = np.random.randint(self.n, size=3)
            self.data[i, j] = - self.data[i, j]
            self.ydata[k] = np.random.rand()
#             time.sleep(.001)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.9 + freq2 * 0.1
            self.tx = f'Simulation steps per second:  {self.freq:.3f}'
            print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
            QtCore.QTimer.singleShot(1, self.simulation_loop)
            self.counter += 1


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

Simulation steps per second:  13333.876

/tmp/ipykernel_305374/1883293687.py:58: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [27]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
        
class ThreadClass(QtCore.QThread):
    def __init__(self, main_window):
        self.main_window = main_window
        super(ThreadClass, self).__init__(main_window)

    def run(self):
        self = self.main_window
        while self.run:
#             print('update')
            i, j, k = np.random.randint(self.n, size=3)
            self.data[i, j] = - self.data[i, j]
            self.ydata[k] = np.random.rand()
#             time.sleep(1)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.99 + freq2 * 0.01
            self.tx = f'Simulation steps per second:  {self.freq:.3f}'
            print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
#             QtCore.QTimer.singleShot(1, self.update)
            self.counter += 1
            
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')
        
        #### Set Data  #####################

        n = self.n = 10
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()
        
        self.update()

        #### Start  #####################
        self.run = True
#         threading.Thread(target=self.simulation_loop).start()
        self.threadclass = ThreadClass(self)
        self.threadclass.start() 
# #         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
    
    def closeEvent(self, event):
        self.run = False

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(self.tx)

if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

/tmp/ipykernel_305374/426323109.py:86: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [1]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
import numba as nb
from numba.experimental import jitclass

def run_simulation(run, data, ydata, tx):
    n, = ydata.shape
    lastupdate = timeit.default_timer()
    freq = 0.
    counter = 0
    while run:
#             print('update')
        i, j, k = np.random.randint(n, size=3)
        data[i, j] = - data[i, j]
        ydata[k] = np.random.rand()
#             time.sleep(1)

        now = timeit.default_timer()
        dt = (now-lastupdate)
        if dt <= 0:
            dt = 0.000000000001
        freq2 = 1.0 / dt
        lastupdate = now
        freq = freq * 0.99 + freq2 * 0.01
        tx = f'Simulation steps per second:  {freq:.3f}'
        print(f'Simulation steps per second:  {freq:.3f}', end='\r')
        counter += 1

class ThreadClass(QtCore.QThread):
    def __init__(self, main_window):
        self.main_window = main_window
        super(ThreadClass, self).__init__(main_window)
    
    def run(self):
        self = self.main_window
        run_simulation(self.run, self.data, self.ydata, self.tx)

class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')
        
        #### Set Data  #####################

        n = self.n = 10
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()
        
        self.update()

        #### Start  #####################
        self.run = np.array(True)
#         threading.Thread(target=self.simulation_loop).start()
        self.threadclass = ThreadClass(self)
        self.threadclass.start() 
# #         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
    
    def closeEvent(self, event):
        np.copyto(self.run, np.array(False))

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(self.tx)

if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

/tmp/ipykernel_311057/2625973819.py:94: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [ ]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
import numba as nb
from numba.experimental import jitclass

timer = timeit.default_timer

@nb.njit
def run_simulation(run, data, ydata, tx):
    n, = ydata.shape
#     lastupdate = time.time()
    freq = 0.
    counter = 0
    while run:
#             print('update')
        i, j, k = [np.random.randint(n) for i in range(3)]
        data[i, j] = - data[i, j]
        ydata[k] = np.random.rand()
#         time.sleep(1)

#         now = time.time()
#         dt = (now-lastupdate)
#         if dt <= 0:
#             dt = 0.000000000001
#         freq2 = 1.0 / dt
#         lastupdate = now
#         freq = freq * 0.99 + freq2 * 0.01
#         str_ = 'Simulation steps per second:  {freq:.3f}'.format(freq=freq)
#         np.copyto(tx, str_)
#         print(str_ + '\r')
        counter += 1

class ThreadClass(QtCore.QThread):
    def __init__(self, main_window):
        self.main_window = main_window
        super(ThreadClass, self).__init__(main_window)
    
    def run(self):
        self = self.main_window
        run_simulation(self.run, self.data, self.ydata, self.tx)

class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')
        
        #### Set Data  #####################

        n = self.n = 10
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = np.array(f'Simulation steps per second: ' + ' ' * 10)
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()
        
        self.update()

        #### Start  #####################
        self.run = np.array(True)
#         threading.Thread(target=self.simulation_loop).start()
        self.threadclass = ThreadClass(self)
        self.threadclass.start() 
# #         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
    
    def closeEvent(self, event):
        np.copyto(self.run, np.array(False))

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(str(self.tx))

if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

/tmp/ipykernel_317225/4099303192.py:98: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [ ]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
        
class ThreadClass(QtCore.QThread):
    def __init__(self, main_window):
        self.main_window = main_window
        super(ThreadClass, self).__init__(main_window)

    def run(self):
        self = self.main_window
        while self.run:
#             print('update')
            i, j, k = np.random.randint(self.n, size=3)
            self.data[i, j] = - self.data[i, j]
            self.ydata[k] = np.random.rand()
            time.sleep(1)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.99 + freq2 * 0.01
            self.tx = f'Simulation steps per second:  {self.freq:.3f}'
            print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
#             QtCore.QTimer.singleShot(1, self.update)
            self.counter += 1
            
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

#         self.canvas = pg.GraphicsLayoutWidget()
#         self.mainbox.layout().addWidget(self.canvas)
        self.canvas = self.graphicsView = pg.GraphicsView()
        self.mainbox.layout().addWidget(self.graphicsView)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

#         self.view = self.canvas.addViewBox()
#         self.view.setAspectLocked(True)
#         self.view.setRange(QtCore.QRectF(0,0, 100, 100))

#         #  image plot
#         self.img = pg.ImageItem(border='w')
#         self.view.addItem(self.img)

#         self.canvas.nextRow()

        vb = pg.ViewBox()
        self.graphicsView.setCentralItem(vb)
        vb.setAspectLocked()
        self.img = pg.ImageItem()
        vb.addItem(self.img)

        #  line plot
#         self.otherplot = self.canvas.addPlot()
#         self.h2 = self.otherplot.plot(pen='y')

        #### Set Data  #####################

        n = self.n = 10000
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()
        
        self.update()

        #### Start  #####################
        self.run = True
#         threading.Thread(target=self.simulation_loop).start()
        self.threadclass = ThreadClass(self)
        self.threadclass.start() 
#         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
    
    def closeEvent(self, event):
        self.run = False
        del self.data
        del self.ydata

    def update(self):
#         print('called _update')
        self.img.setImage(self.data, autoLevels=False)
#         self.h2.setData(self.ydata)
        self.label.setText(self.tx)

#     def simulation_loop(self):
#         while self.run:
# #             print('update')
#             i, j, k = np.random.randint(self.n, size=3)
#             self.data[i, j] = - self.data[i, j]
#             self.ydata[k] = np.random.rand()
# #             time.sleep(.001)

#             now = timeit.default_timer()
#             dt = (now-self.lastupdate)
#             if dt <= 0:
#                 dt = 0.000000000001
#             freq2 = 1.0 / dt
#             self.lastupdate = now
#             self.freq = self.freq * 0.9 + freq2 * 0.1
#             self.tx = f'Simulation steps per second:  {self.freq:.3f}'
#             print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
# #             QtCore.QTimer.singleShot(1, self.update)
#             self.counter += 1


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()
    del app

/tmp/ipykernel_299139/569687140.py:95: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [ ]:
import time
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QWidget, QVBoxLayout, QProgressBar, QPushButton
import multiprocessing
from multiprocessing import shared_memory
import numpy as np

DATA_SIZE = 1
DTYPE = np.int64

context = multiprocessing.get_context("fork")


class DataPuller(context.Process):
    def __init__(
        self, event, shared_mem_name,
    ):
        super().__init__()
        self.event = event
        self.shared_mem_name = shared_mem_name

    def run(self):
        self.shm = shared_memory.SharedMemory(name=self.shared_mem_name)
        self.sample = np.ndarray(shape=(DATA_SIZE, 1), dtype=DTYPE, buffer=self.shm.buf)
        self.counter = 0

        while not self.event.is_set():
            time.sleep(0.1)
            self.sample[:, 0] = self.counter
            print("child: ", self.sample[0, 0])
            self.counter += 1
            if self.counter >= 1000:
                self.counter = 0
        self.shm.close()
        print("child process finished")


class Window(QWidget):
    def __init__(self, *args, **kwargs):
        super(Window, self).__init__(*args, **kwargs)
        layout = QVBoxLayout(self)
        self.progressBar = QProgressBar()
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.onTimer)
        self.progressBar.setRange(0, 1000)
        layout.addWidget(self.progressBar)
        layout.addWidget(QPushButton("开启线程", clicked=self.onStart))

        self.counter = 0
        self.shared_mem_name = "data"
        self.base_array = np.zeros((DATA_SIZE, 1), dtype=DTYPE)
        self.shm = shared_memory.SharedMemory(
            create=True, size=self.base_array.nbytes, name=self.shared_mem_name
        )
        self.sample = np.ndarray(
            shape=self.base_array.shape,
            dtype=self.base_array.dtype,
            buffer=self.shm.buf,
        )
        self.event = context.Event()
        print(self.event.is_set())
        self._process = DataPuller(self.event, self.shared_mem_name)

    def onStart(self):
        if not self._process.is_alive():
            print("main starting process")
            self._process.start()
            self.timer.start(500)
        else:
            pass

    def onTimer(self):
        print("main: ", self.sample[0, 0])
        self.progressBar.setValue(self.sample[0, 0])

    def closeEvent(self, event):
        if self._process.is_alive():
            self.event.set()
            self._process.join()
        self.shm.close()
        self.shm.unlink()
        self.close()
        print("main process finished")


if __name__ == "__main__":
    import sys
    from PyQt5.QtWidgets import QApplication

    app = QApplication(sys.argv)
    w = Window()
    w.show()
    sys.exit(app.exec_())


False
main starting process
child:  0
child:  1
child:  2
child:  3
child:  4
main:  3
child:  5
child:  6
child:  7
child:  8
child:  9
main:  7
child:  10
child:  11
child:  12
child:  13
main:  11
child:  14
child:  15
child:  16
child:  17
child:  18
main:  16
child:  19
child:  20
child:  21
child:  22
child:  main:  21
23
child:  24
child:  25
child:  26
child:  27
main:  25
child:  28
child:  29
child:  30
child:  31
child:  32
main:  30
child:  33
child:  34
child:  35
child:  36
child:  37
main:  35
child:  38
child:  39
child:  40
child:  41
main:  40
child:  42
child:  43
child:  44
child:  45
child:  46
main:  44
child:  47
child:  48
child:  49
child:  50
main:  49
child:  51
child:  52
child:  53
child process finished


https://stackoverflow.com/questions/60580274/multiprocessing-shared-memory-in-pyqt5-python-3-8

In [1]:
import sys
import time
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
import threading
import timeit
import multiprocessing
from multiprocessing import shared_memory


context = multiprocessing.get_context("spawn")

class Worker(context.Process):
    def __init__(self, event, shared_mem_name, dtype, shape):
        super().__init__()
        self.event = event
        self.shared_mem_name = shared_mem_name
        self.dtype = dtype
        self.shape = shape
    
    def run(self):
        self.shm = shared_memory.SharedMemory(name=self.shared_mem_name)
        self.sample = np.ndarray(shape=self.shape, dtype=self.dtype, buffer=self.shm.buf)
        self.counter = 0
        self.freq = 0.
        self.data = self.sample
        n, n = self.shape
        self.lastupdate = timeit.default_timer()
        self.data[:, :] = np.random.randint(2, size=(n, n), dtype=self.dtype)

        while not self.event.is_set():
#             print('update')
            i, j, k = np.random.randint(n, size=3)
            self.data[i, j] = 1#- self.data[i, j]
#             self.ydata[k] = np.random.rand()
#             time.sleep(.1)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.9 + freq2 * 0.1
            print(f'\rSimulation steps per second:  {self.freq:.3f}', end='\r')
#             QtCore.QTimer.singleShot(1, self.update)
            self.counter += 1
        self.shm.close()
        
        
    
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        n = self.n = 10
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.shared_mem_name = "data"
        self.base_array = self.data
        self.shm = shared_memory.SharedMemory(
            create=True, size=self.base_array.nbytes, name=self.shared_mem_name
        )
        self.sample = np.ndarray(
            shape=self.base_array.shape,
            dtype=self.base_array.dtype,
            buffer=self.shm.buf,
        )
        self.event = context.Event()
        self._process = Worker(self.event, self.shared_mem_name, self.base_array.dtype, self.base_array.shape)

        if not self._process.is_alive():
            self._process.start()

        self.data = self.sample

        #### Start  #####################
#         threading.Timer(1 / 60, self._update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self._update)
        timer.start(1 / 60 * 1000)

    def closeEvent(self, event):
#         if self._process.is_alive():x
        self.event.set()
        self._process.join()
        self.shm.close()
        self.shm.unlink()
        self.close()

    def _update(self):
#         print('start')
#         i, j, k = np.random.randint(self.n, size=3)
#         self.ydata[k] = np.random.rand()
        self.img.setImage(self.data)
#         print(self.data)
#         self.h2.setData(self.ydata)
#         self.label.setText(self.tx)
#         print('end')


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

/tmp/ipykernel_241293/3771611516.py:112: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/alz/miniconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/alz/miniconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'Worker' on <module '__main__' (built-in)>


In [5]:
import sys
import time
import timeit
import threading
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
        
    
class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        n = self.n = 1000
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = 2 * np.random.randint(2, size=(n, n)) - 1
        self.ydata = np.random.rand(n)

        self.tx = ''
        self.counter = 0
        self.freq = 0.
        self.lastupdate = timeit.default_timer()

        #### Start  #####################
        self.run = True
        threading.Thread(target=self.simulation_loop).start()
#         threading.Timer(1 / 60, self.update).start()
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update)
        timer.start(1 / 60 * 1000)
    
    def closeEvent(self, event):
        self.run = False

    def update(self):
#         print('called _update')
        self.img.setImage(self.data)
        self.h2.setData(self.ydata)
        self.label.setText(self.tx)

    def simulation_loop(self):
        while self.run:
#             print('update')
            i, j, k = np.random.randint(self.n, size=3)
            self.data[i, j] = - self.data[i, j]
            self.ydata[k] = np.random.rand()
#             time.sleep(.001)

            now = timeit.default_timer()
            dt = (now-self.lastupdate)
            if dt <= 0:
                dt = 0.000000000001
            freq2 = 1.0 / dt
            self.lastupdate = now
            self.freq = self.freq * 0.9 + freq2 * 0.1
            self.tx = f'Simulation steps per second:  {self.freq:.3f}'
            print(f'Simulation steps per second:  {self.freq:.3f}', end='\r')
#             QtCore.QTimer.singleShot(1, self.update)
            self.counter += 1


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

Simulation steps per second:  15349.822

/tmp/ipykernel_270134/3930852360.py:58: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  timer.start(1 / 60 * 1000)


In [ ]:

self.graphicsView = GraphicsView()
vb = pg.ViewBox()
self.graphicsView.setCentralItem(vb)
vb.setAspectLocked()
img = pg.ImageItem()
vb.addItem(img)

In [15]:
1 / 60 * 1000

33.333333333333336

In [1]:
import sys
import time
from pyqtgraph.Qt import QtCore, QtGui, QtWidgets
import numpy as np
import pyqtgraph as pg
import numba as nb
[]

class App(QtGui.QMainWindow):
    def __init__(self, parent=None):
        super(App, self).__init__(parent)

        #### Create Gui Elements ###########
        self.mainbox = QtGui.QWidget()
        self.setCentralWidget(self.mainbox)
        self.mainbox.setLayout(QtGui.QVBoxLayout())

        self.canvas = pg.GraphicsLayoutWidget()
        self.mainbox.layout().addWidget(self.canvas)

        self.label = QtGui.QLabel()
        self.mainbox.layout().addWidget(self.label)

        self.view = self.canvas.addViewBox()
        self.view.setAspectLocked(True)
        self.view.setRange(QtCore.QRectF(0,0, 100, 100))

        #  image plot
        self.img = pg.ImageItem(border='w')
        self.view.addItem(self.img)

        self.canvas.nextRow()
        #  line plot
        self.otherplot = self.canvas.addPlot()
        self.h2 = self.otherplot.plot(pen='y')


        #### Set Data  #####################

        n = 10000
        self.x = np.linspace(0, 50., n)
        self.X,self.Y = np.meshgrid(self.x,self.x)
        self.data = np.empty((n, n))
        self.ydata = np.empty(n)
        self.datas = np.random.rand(2, n, n) < 0.5
        self.ydatas = np.random.rand(2, n)  < 0.5

        self.counter = 0
        self.fps = 0.
        self.lastupdate = time.time()

        #### Start  #####################
#         self._update_val(self.data, self.ydata, self.x, self.X, self.Y, self.counter)
        self._update()

    @staticmethod
    @nb.jit(nopython=True)
    def _update_val(data, ydata, x, X, Y, counter):
        data[:] = np.sin(X/3.+counter/9.)*np.cos(Y/3.+counter/9.)
        ydata[:] = np.sin(x/3.+ counter/9.)

#     @staticmethod
# #     @nb.jit(nopython=True)
#     def _update_val1(data, ydata):
#         data[:] = np.roll(data, (1, 1), axis=(0, 1))
#         ydata[:] = np.roll(ydata, 1)

    @staticmethod
#     @nb.jit(nopython=True)
    def _update_val1(data, ydata):
        n, = ydata.shape
        data[:] = np.random.rand(n, n)
        ydata[:] = np.random.rand(n)

    def _update(self):
#         self._update_val1(self.data, self.ydata)

        i = self.counter % 2
        self.img.setImage(self.datas[i])
        self.h2.setData(self.ydatas[i])

        now = time.time()
        dt = (now-self.lastupdate)
        if dt <= 0:
            dt = 0.000000000001
        fps2 = 1.0 / dt
        self.lastupdate = now
        self.fps = self.fps * 0.9 + fps2 * 0.1
        tx = 'Mean Frame Rate:  {fps:.3f} FPS\n1'.format(fps=self.fps )
        self.label.setText(tx)
        QtCore.QTimer.singleShot(1, self._update)
        self.counter += 1


if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    thisapp = App()
    thisapp.show()
    app.exec_()

In [4]:
import numpy as np
counter = 1
x = np.linspace(0,50., num=10000)
X, Y = np.meshgrid(x, x)
data = np.sin(X/3+counter/9.)*np.cos(Y/3.+counter/9.)
ydata = np.sin(x/3.+ counter/9.)

In [2]:
import numpy as np

class self:
    pass

self = self()
n = 10000
self.x = np.linspace(0, 50., n)
self.X,self.Y = np.meshgrid(self.x,self.x)
self.data = np.empty((n, n))
self.ydata = np.empty(n)

In [7]:
data.nbytes * 10 ** -6
ydata.nbytes * 10 ** -6

0.08

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


def update_title(axes):
    axes.set_title(datetime.now())
    axes.figure.canvas.draw()

fig, ax = plt.subplots()

x = np.linspace(-3, 3)
ax.plot(x, x ** 2)

# Create a new timer object. Set the interval to 100 milliseconds
# (1000 is default) and tell the timer what function should be called.
timer = fig.canvas.new_timer(interval=1 / 60)
timer.add_callback(update_title, ax)
timer.start()

# Or could start the timer on first figure draw
#def start_timer(event):
#    timer.start()
#    fig.canvas.mpl_disconnect(drawid)
#drawid = fig.canvas.mpl_connect('draw_event', start_timer)

plt.show()
